In [70]:
import urllib.parse
import urllib.request
import pickle
import re
import numpy as np

[2, 1, 0]


In [71]:
#я простой блок для проверки методов над матрицами. не смотрите на меня, я тут случайно
    #      F I O
result = [[0,0,1],
          [0,1,0],
          [1,0,0]]
print(GetOrder(result))

words = ["Бучнева", "Анна", "Всеволодовна"]

[2, 1, 0]


In [72]:
path_to_names = "D:/"
path_to_patronymics = "D:/"

with open(path_to_names+"NamesFromGufo.pickle","rb") as f:
    tmpNames=pickle.load(f)
with open(path_to_patronymics+"PatronymicsFromGufo.pickle","rb") as f:
    tmpPatronymics=pickle.load(f)    

all_names=[]
all_patronymics=[]

for i in tmpNames:
    all_names.append(i)
for i in tmpPatronymics:
    all_patronymics.append(i)

In [73]:
def NotBruteAtAll(tmp): #это будущий общий метод, в котором будут вызываться все следующие
    res = []
    
    for t in temp:
        result = [] #ответ в шаблоне ФИО
        
        resultStrict = StrictCheck(t)
        if checkMatrix(resultStrict):
            order = GetOrder(result)
            for i in range(3):
                result.append(words[order[i]])   
        # + установить, что если частота меньше N проверить реплейсом (т.к. только редкие, э-экономия)
        else:
            resultRepl = ReplaceCheck(t)
            resultReplValues = resultRepl.values()
            
            resultGramma = GrammaCheck(t) 
            #Сделать грамма чек если только все остальное пусто 
            # ИЛИ мы сравнили строгий результат и гибкий, и там все непонятно
            #Потому что иначе, если уже найдено, то зачем проверять: если ошибка в написании, она может быть
            resultStatistics = statistics
            
        #result = np.array(result)
        #подсчитываем все с матрицами
        #принимаем решение, записываем в рез
        #вызываем метод SetStatistics и обновляем статистику порядка
        
        res.append(result)
    return res

In [74]:
def GetOrder(matrix):
    order = [0,0,0]
    for j in range(3):
        for i in range(3):
            if (matrix[i][j]!=0):
                order[j] = i
                break
    return order

In [75]:
def GetOrderAndCheck(matrix):
    order = [None,None,None]
    for j in range(3):
        for i in range(3):
            if (matrix[i][j]!=0):
                if (i not in order):
                    order[j] = i
                    break
                else: return False
        if order[j] == None: return False
    return order

In [76]:
def checkMatrix(matrix): #Если у нас в одной строке и столбце только по одному элементу, то возвращает True
    for i in range(3):
        k1 = 0
        if (matrix[i] != 0):
            k1 += 1
            k2 = 0
            for j in range(3):
                if (matrix[i][j] != 0): k2+=1
            if (k2 != 1):
                return False
        if (k1 != 1):
            return False
    return True

In [77]:
def GetMaxOrder(matrix):
    order = [None,None,None]
    for j in range(3):
        maxI = 0
        for i in range(3):
            if (matrix[i][j]>matrix[maxI][j]):
                maxI = i
        if (maxI not in order):
            order[j] = maxI
        else: return False
    return order

In [78]:
def StrictCheck(temp): #строгая проверка
    temp = temp.lower()
    words = temp.split(" ")
    #          F I O
    result = [[0,0,0],
              [0,0,0],
              [0,0,0]]
    
    for i in range(3):
        w = words[i].strip()
        
        # ХДЕ БАЗА ФАМИЛИЙ 
        if (w in all_surnames): 
            result[i][0] = surnamesFrequency[w] 
            #пока делаю из расчета на то что у нас есть словарь с частотами      
        if (w in all_names):
            result[i][1] = namesFrequency[w]          
        if (w in all_patronymics):
            result[i][2] = patronymicsFrequency[w] 
            
    return result

In [79]:
def ReplaceCheck(temp): #проверка с перестановками
    temp = temp.lower()
    words = temp.split(" ")
    #          F I O
    result = [[0,0,0],
              [0,0,0],
              [0,0,0]]

    for i in range(3):
        w = words[i].strip()
        
        result[i][0] = forReplaceCheck(w, all_surnames)
        result[i][1] = forReplaceCheck(w, all_names)           
        result[i][2] = forReplaceCheck(w, all_patronymics) 
    #В МАТРИЦЕ НЕ ПРОСТО ВЕРОЯТНОСТИ, А СЛОВАРЬ = СЛОВО: ВЕРОЯТНОСТЬ, ПОЭТОМУ СРАВНИВАТЬ ПО VALUE    
    return result

In [80]:
def forReplaceCheck(w, list): #выделила в отдельный метод, т.к. иначе приходилось три раза одно и тоже писать. Потна
    res = [{},{}]
    for l in list:
        dist = shortDistance(w, l)
        if (dist != False):
            res[int(dist)-1][l] = 1 #frequency*10**(-2*dist)
    #проверить почему при поиске Анны выдает две Юноны?
    r1 = max(res[0])
    r2 = max(res[1])
    return max(r1, r2)

In [81]:
def shortDistance(word, dictWord): #упрощенное расстояние (поиск только с опечаткой 1 и 2)
    #word - слово которое проверяем
    #dictWord - слово из списка, с которым сравниваем
    if word == dictWord: return 0
    elif len(word) == 0: return len(dictWord)
    elif len(dictWord) == 0: return len(word)
    elif len(dictWord) > len(word)+2 and len(dictWord) < len(word)+2:
        return False
    dist = levenshtein(word, dictWord)
    if (dist == 1 or dist==2):
        return dist
    else: return False

In [82]:
def levenshtein(s, t): #расстояние Левенштейна
    if s == t: return 0
    elif len(s) == 0: return len(t)
    elif len(t) == 0: return len(s)
    v0 = [None] * (len(t) + 1)
    v1 = [None] * (len(t) + 1)   
    for i in range(len(v0)):
        v0[i] = i
    for i in range(len(s)):
        v1[0] = i + 1
        for j in range(len(t)):
            cost = 0 if s[i] == t[j] else 1
            v1[j + 1] = min(v1[j] + 1, v0[j + 1] + 1, v0[j] + cost)
        for j in range(len(v0)):
            v0[j] = v1[j]
    return v1[len(t)]

In [83]:
def GrammaCheck(temp): #проверка по грамматике
    temp = temp.lower()
    words = temp.split(" ")
    #          F I O
    result = [[0,0,0],
              [0,0,0],
              [0,0,0]]

    for i in range(3):
        w = words[i].strip()
        result[i][0] = checkSurnames(w)
        result[i][2] = checkPatronymic(w)

    k = 0
    for i in range(3):
        flag = True
        for j in range(3):
            if (result[i][j] != 0): 
                flag = False 
                break
        if(flag):
            result[i][1] = 1
            k+=1
    if(k==0):
        for i in range(3): result[i][1] = 0.33
    elif(k>1): 
        for i in range(3): result[i][1] = result[i][1]/k
    return result

In [84]:
def checkSurnames(s):#раньше возвращало тру фолс, теперь 1 или 0
    pattern = '\w*(ов|ова|ев|ёв|ева|ёва|ив|ин|ина|ын|их|ых|ский|цкий|ая|ко|дзе|онок|ян)$'
    if (re.match(pattern,s)): return 1
    else: return 0

In [85]:
def checkPatronymic(s):
    pattern = '\w*(ович|евич|ич|овна|евна|ична|инична)$'
    if (re.match(pattern,s)):
        return 1
    else: return 0

In [86]:
#               F     I    O
statistics = [[0.50,0.45,0.05],
              [0.10,0.50,0.40],
              [0.40,0.05,0.55]]
# FIO: 0.50, IOF: 0.40, IFO: 0.15, ElseOptions: 0.05
#пока что совсем от балды, простите
def SetStatistics(order):
    for i in range(3):
        for j in range(3):
            if (j == order[i]): 
                statistics[i][j] += 0.002
            else: statistics[i][j] -= 0.001
#Обновляет статистику с учетом результата выполнения алгоритма            

In [87]:
def Test(): 
    with open("D:/test_changed.pickle","rb") as f:
        tmp=pickle.load(f)
    
    testing_res=NotBruteEnough(tmp)
    for fio in testing_res:
        print(fio)

    errs=0
    for i in testing_res:
        if (i.find("%")!=-1):
            print (i)
            errs+=1
    print("Всего не уверены в: " + str(errs))

In [88]:
def NotBruteEnough(tmp):
    res = []

    for t in tmp:
        name = ""
        surname = ""
        patronymic = ""
        t = t.lower()
        words = t.split(" ")

        for w in words:
            w.strip()
            if (name == ""):
                if (w in all_names):
                    name = w

            if (patronymic == ""):
                if (w in all_patronymics):
                    patronymic = w

            if (name != w and patronymic != w and checkSurnames(w) and surname == ""):
                surname = w

        if (name in words): words.remove(name)
        if (patronymic in words): words.remove(patronymic)
        if (surname in words): words.remove(surname)

        if (name == "") & (len(words) > 0):
            foundWord = ""
            for w in words:
                minDist = len(w)
                for n in all_names:
                    dist = levenshtein(w, n)
                    if dist < minDist:
                        minDist = dist
                        name = n.title() #+ " (" + str(round((1 - minDist / len(w)) * 100)) + "% " + w + ")"
                        foundWord = w
            words.remove(foundWord)

        if (patronymic == "") & (len(words) > 0):
            foundWord = ""
            for w in words:
                minDist = len(w)
                for p in all_patronymics:
                    dist = levenshtein(w, p)
                    if dist < minDist:
                        minDist = dist
                        patronymic = p.title() #+ " (" + str(round((1 - minDist / len(w)) * 100)) + "% " + w + ")"
                        foundWord = w
            words.remove(foundWord)

        if (surname == "")& (len(words) > 0):
            surname = words[0]

        name = name.title()
        surname = surname.title()
        patronymic = patronymic.title()
        res.append(surname + " " + name + " " + patronymic)

    return res

In [89]:
#genders Пока не используем
genderTuple = ('male', 'female', 'unknown')

def checkSurnamesGender(s):
    patternMale = '\w*(ов|ев|ий|ын|ин)$'
    patternFem ='\w*(ова|ева|ая|ина|ына)$'
    patternUnknown = '\w*(их|ых|ко|ук|юк|ун|ний|ный|чай|ий|а|ич|ов|ук|ик|ски|ка|ски|цки|дзки)$'
    
    if (re.match(patternMale,s)):
        return genderTuple[0]
    elif (re.match(patternFem,s)):
        return genderTuple[1]
    elif (re.match(patternUnknown,s)):
        return genderTuple[2]
    else: return 'No'

def changeGenderSurname(s):
    patternMaleA = '\w*(ов|ев|ин|ын)$'
    patternFemA ='\w*(ова|ева|ина|ына)$'
    patternMaleB = '\w*(ий)$'
    patternFemB = '\w*(ая)$'
    
    if(re.match(patternMaleA, s)):
        return s + 'a'
    elif (re.match(patternMaleB, s)):       
        s += ' '
        return s.replace('ий ', 'ая')
    elif (re.match(patternFemA, s)):
        return s[0: len(s)-1]
    elif (re.match(patternFemB, s)): 
        s += ' '
        return s.replace('ая ', 'ий')
    else: return s

def checkPatronymicGender(s):
    patternMale = '\w*(ович|евич|ич)$'
    patternFem ='\w*(овна|евна|ична|инична)$'
   
    if (re.match(patternMale,s)):
        return genderTuple[0]
    elif (re.match(patternFem,s)):
        return genderTuple[1]
    else: return 'No'    

def changeGenderPatronymic(s):
    patternMaleA = '\w*(ович|евич)$'
    patternFemA ='\w*(овна|евна)$'
    patternMaleB = '\w*(ич)$'
    patternFemB = '\w*(ична|инична)$'

    if(re.match(patternMaleA, s)):
        s += ' '
        return s.replace('ич ', 'на')
    if(re.match(patternFemA, s)):
        s += ' '
        return s.replace('на ', 'ич')     
    if(re.match(patternMaleB, s)):
        return s + 'на'   
    if(re.match(patternFemB, s)):
        s += ' '
        s = s.replace('инична ', 'ич')
        s = s.replace('ична ', 'ич')
        return s    
    else: return s
